In [ ]:
import sys
import pandas as pd

# Imports 

In [ ]:
user = pd.read_csv('/content/drive/MyDrive/Project/Dataset/merged data/user.csv')
tweet = pd.read_csv('/content/drive/MyDrive/Project/Dataset/merged data/tweet.csv')

In [ ]:
temp=user[user.duplicated(['userId'])]
temp.shape[0]

4854

In [ ]:
%cd /content/drive/MyDrive/rumor_detection/

/content/drive/MyDrive/rumor_detection


In [ ]:
%ls -l

total 462423
-rw------- 1 root root 297420892 Mar 19 05:13 data.csv
-rw------- 1 root root 157189018 May 10 05:20 meta_data.sav
-rw------- 1 root root      3686 May 10 04:38 meta_utils.py
drwx------ 2 root root      4096 Apr  7 17:03 __pycache__/
-rw------- 1 root root  14848184 Mar 19 05:13 reactions_data.csv
-rw------- 1 root root   2532919 Mar 19 05:13 source_tweet.csv
drwx------ 2 root root      4096 May 10 07:11 tweet_user_checkpoint/
-rw------- 1 root root   1504564 Mar 19 05:12 user.csv
-rw------- 1 root root     11727 Apr  8 15:06 utils.py


In [ ]:
tweet.drop_duplicates(keep='first',inplace=True)

# Creating test dataset

In [ ]:
#Live dataset input here

# Tweet and user meta data model 1


In [ ]:
from meta_utils import *

In [ ]:
copy_tweet_df=tweet
copy_user_df=user

In [ ]:
def create_numof_occurences_column(df,dct):
  for i in range(0,df.shape[0]):
    tweet_id=df['tweetId'][i]
    user_id=df['userId'][i]
    if ((tweet_id,user_id) in dct.keys()):
      dct[(tweet_id,user_id)]=dct[(tweet_id,user_id)]+1
      num_of_occurence=dct[(tweet_id,user_id)]
    else:
      dct[(tweet_id,user_id)]=1
      num_of_occurence=dct[(tweet_id,user_id)]
    df['occurence_number'][i]=num_of_occurence

def merged_df(user_df,tweet_df): #User_data_df and tweet_data_df as input
    #tweet.drop_duplicates(keep='first',inplace=True)
    empty_dict_user={}
    empty_dict_tweet={}
    user_df['occurence_number']=""
    tweet_df['occurence_number']=""
    create_numof_occurences_column(user,empty_dict_user)
    create_numof_occurences_column(tweet,empty_dict_tweet)
    user_df[['day','month','date','hh:mm:ss','idk','year']]=user_df['created_at'].str.split(" ",expand=True)
    user_df[['hour','minute','second']]=user_df['hh:mm:ss'].str.split(":",expand=True)
    import datetime
    for i in range(0,user.shape[0]):
        user_df['month'][i] = (datetime.datetime.strptime(user_df['month'][i], "%b")).month
    user_df['year']=user_df['year'].astype(int)
    user_df['hour']=user_df['hour'].astype(int)
    user_df['second']=user_df['second'].astype(int)
    user_df['date']=user_df['date'].astype(int)
    user_df['minute']=user_df['minute'].astype(int)
    user_df['month']=user_df['month'].astype(int)
    user_df['account_age']=""
    from datetime import date
    today = date.today()
    current_date = today.strftime("%d/%m/%Y")
    current_date
    from dateutil import parser
    current_date=parser.parse(current_date)
    current_date.month,current_date.year,current_date.day
    from datetime import date
    d1 =date(current_date.year,current_date.month,current_date.day)
    for i in range(0,user_df.shape[0]):
        temp1=date(user_df['year'][i],user_df['month'][i],user_df['date'][i])
        user_df['account_age'][i]=d1-temp1
    user['account_age_in_years']=""
    for i in range(0,user.shape[0]):
        user_df['account_age_in_years'][i]=(user_df['account_age'][i].days)/365
    user_df.drop(['day','month','date','hh:mm:ss','idk','year','hour','minute','second'],axis=1,inplace=True)
    merged_data = pd.merge(user,tweet,on=['userId','tweetId','occurence_number'],how='inner')
    merged_data.drop(['name','description','screen_name','url','text','account_age'],inplace=True,axis=1)
    merged_data['labels']=merged_data['labels'].replace(['unverified'],True)
    merged_data['labels']=merged_data['labels'].replace(['TRUE'],True)
    merged_data['labels']=merged_data['labels'].replace(['FALSE'],True)
    merged_data['labels']=merged_data['labels'].replace(['non-rumor'],False)
    merged_data['labels']=merged_data['labels'].replace(['true'],True)
    merged_data['labels']=merged_data['labels'].replace(['false'],True)
    merged_data['labels']=merged_data['labels'].astype(int)
    return merged_data

def tt_split(merged_data):
    from sklearn.model_selection import train_test_split
    x_columns=['followers_count', 'statuses_count', 'friends_count','account_age_in_years', 'favorite_count']
    X_train,X_test,y_train,y_test=train_test_split(merged_data[x_columns],
    merged_data['labels'],
    test_size=0.3,
    random_state=42,stratify=merged_data['labels'])
    return X_train,X_test,y_train,y_test

def test_data(merged_data):
    X_test = merged_data.drop(labels=['userId','tweetId','location','created_at_x','occurence_number','source','entities_symbols','entities_user_mentions','entities_hashtags','entities_url','created_at_y','labels'], axis=1)
    y_test = merged_data['labels']
    return X_test,y_test



In [ ]:
merged_data = merged_df(copy_user_df,copy_tweet_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launc

In [ ]:
merged_data[merged_data.duplicated()]

,userId,tweetId,verified,followers_count,statuses_count,friends_count,location,created_at_x,occurence_number,account_age_in_years,favorite_count,retweet_count,source,entities_symbols,entities_user_mentions,entities_hashtags,entities_url,created_at_y,n_symbols,n_urls,n_hashtags,n_user_mentions,labels


In [ ]:
X_train,X_test,y_train,y_test = tt_split(merged_data)

In [ ]:
X_train.head()

,followers_count,statuses_count,friends_count,account_age_in_years,favorite_count
4820,3380,46520,2075,11.9589,231
7718,8723,59570,983,10.0548,138
65,72203,133120,8674,13.6384,60
6395,9968,24181,1695,10.9973,58
4147,98299,17388,808,12.2219,6


In [ ]:
X_test.head()

,followers_count,statuses_count,friends_count,account_age_in_years,favorite_count
4964,7945684,142412,1844,10.4986,391
230,533,2423,789,8.53699,77
6895,56870,1396,43061,8.56986,577
7352,20158591,424816,260,14.1699,295
4854,22764,80687,990,12.3616,46


In [ ]:
X_test.shape[0]

2459

In [ ]:
y_train.head()

4820    0
7718    1
65      1
6395    0
4147    1
Name: labels, dtype: int64

In [ ]:
import joblib
loaded_model = joblib.load('meta_data.sav')
model1_predict_probabs = loaded_model.predict_proba(X_test)
result = loaded_model.score(X_test, y_test)

In [ ]:
#Column1 is probabilty of zero, column2 is probability of one

model1_predict_probabs

array([[0.079, 0.921],
       [0.865, 0.135],
       [0.321, 0.679],
       ...,
       [0.818, 0.182],
       [0.067, 0.933],
       [0.724, 0.276]])

In [ ]:
print(result)

0.9133794225294836


# Model 2 ( User profile model )

In [ ]:
from ast import literal_eval
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, StandardScaler


def tweet_meta_data_util(tweet_data, user_data):

  data = tweet_data.merge(user_data[['tweetId', 'created_at']], on='tweetId', suffixes=['_tweet', '_user'], how='inner')
  data['created_at_tweet'] = data['created_at_tweet'].apply(lambda date_time_str : datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S'))
  data['created_at_user'] = data['created_at_user'].apply(lambda date_time_str : datetime.strptime(date_time_str, '%a %b %d %H:%M:%S +0000 %Y'))
  data['n_days'] = data['created_at_tweet'] - data['created_at_user']
  data['n_days'] = data['n_days'].apply(lambda x : x.days)
  data['labels'] = data['labels'].astype('int')
  data = data[['favorite_count','retweet_count','n_symbols','n_urls','n_hashtags','n_user_mentions', 'n_days','labels' ]]

  X = data[['retweet_count','n_urls','n_hashtags','n_user_mentions', 'n_days']]
  y = data['labels']

  return X, y


def user_tweet_meta_util(tweet_data, user_data):
  user_data['verified'] = user_data['verified'].map({True:1, False:0})
  user_unique = pd.DataFrame(user_data.groupby('userId').mean())
  user_unique['verified']  = user_unique['verified'].apply(lambda x: x > 0.5 )
  user_unique = user_unique.reset_index()
  tweet_data['labels'] = tweet_data['labels'].map({'true':1, 'false':1, 'unverified':1, 'non-rumor':0})
  x = pd.pivot_table(tweet, values='tweetId', columns=['labels'], index=['userId'], fill_value=0, aggfunc="count")
  x = x.reset_index()
  data = user_unique.merge(x, on='userId')
  data = data.rename(columns={0:"non-rumor", 1:"rumor"})
  data['labels'] = np.where(data['non-rumor'] >= data['rumor'], 'non-rumor', 'rumor')
  data.drop([ 'userId', 'rumor', 'non-rumor'], axis=1, inplace=True)
  le = LabelEncoder()
  data['verified'] = le.fit_transform(data['verified'])
  le_labels = LabelEncoder()
  data['labels'] = le_labels.fit_transform(data['labels'])
  scaler = StandardScaler()
  data[['followers_count', 'statuses_count', 'friends_count']] = scaler.fit_transform(data[['followers_count', 'statuses_count', 'friends_count']])
  X = data[['verified', 'followers_count', 'statuses_count', 'friends_count']]
  y = data['labels']
  return X, y

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/Rumor Detection/models/user_meta_model')

In [ ]:
tweet.head()

,tweetId,text,userId,favorite_count,retweet_count,source,entities_symbols,entities_user_mentions,entities_hashtags,entities_url,created_at,n_symbols,n_urls,n_hashtags,n_user_mentions,labels,occurence_number
0,500339512082698240,From jaywalking to strong-arm robbery. \n\nMhm...,2725579568,139,213,Twitter for iPhone,[],[],"[{'indices': [48, 57], 'text': 'Ferguson'}]",[],2014-08-15 17:53:30,0,0,1,0,1,1
1,581386155083620352,Andreas Lubitiz was 'deemed unsuitable for fli...,16973333,7,30,TweetDeck,[],[],"[{'indices': [75, 87], 'text': 'Germanwings'}]","[{'url': 'http://t.co/nk5GGyuxaa', 'indices': ...",2015-03-27 09:23:55,0,1,1,0,1,1
2,499527157023277056,"Classes started at 8am, but by 7am #MichaelBro...",14849562,140,243,Twitter for iPhone,[],[],"[{'indices': [35, 48], 'text': 'MichaelBrown'}...","[{'url': 'http://t.co/SVaqe4vR2V', 'indices': ...",2014-08-13 12:05:29,0,1,2,0,0,1
3,552829971631472640,French MP says those who attacked Charlie Hebd...,786764,77,132,SocialFlow,[],[],[],"[{'url': 'http://t.co/77d5dBBptH', 'indices': ...",2015-01-07 14:11:50,0,1,0,0,0,1
4,498304114955419648,Protest continues in front of #ferguson police...,27801824,76,215,Twitter for iPhone,[],[],"[{'indices': [30, 39], 'text': 'ferguson'}, {'...",[],2014-08-10 03:05:33,0,0,2,0,0,1


In [ ]:
import numpy as np
X,y=user_tweet_meta_util(tweet,user)

In [ ]:
model.summary()

Model: "sequential_74"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_187 (Dense)            (None, 8)                 40        
_________________________________________________________________
dense_188 (Dense)            (None, 4)                 36        
_________________________________________________________________
dropout_39 (Dropout)         (None, 4)                 0         
_________________________________________________________________
dense_189 (Dense)            (None, 1)                 5         
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.model_selection import train_test_split

X_train_model2,X_test_model2,y_train_model2,y_test_model2=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [ ]:
X_train_model2

,verified,followers_count,statuses_count,friends_count
2105,0,-0.199257,-0.492573,-0.114507
968,1,-0.193344,-0.420747,-0.089400
1113,1,-0.199213,-0.553598,-0.096530
2555,0,-0.199231,0.196701,-0.097841
2781,0,-0.149875,3.214770,-0.048129
...,...,...,...,...
1093,1,-0.199869,-0.550953,-0.104324
2817,0,-0.199198,-0.191437,-0.081605
1393,0,-0.198955,-0.326920,-0.107790
395,0,-0.198647,1.563919,-0.088430


In [ ]:
result_model2 = model.evaluate(X_test_model2,y_test_model2,verbose=2)

32/32 - 0s - loss: 0.6523 - accuracy: 0.6271


In [ ]:
y_test_model2,y_test

(1137    1
 594     1
 283     1
 1624    0
 2750    0
        ..
 3075    0
 1906    0
 2973    0
 1633    0
 2130    1
 Name: labels, Length: 1003, dtype: int64, 4964    1
 230     0
 6895    1
 7352    1
 4854    0
        ..
 2987    1
 5287    1
 7002    0
 935     1
 9       0
 Name: labels, Length: 2459, dtype: int64)

In [ ]:
y_test.shape[0]

2459

In [ ]:
y_test_model2.shape[0]

1003

In [ ]:
model2_predict_probabs = model.predict_proba(X_test_model2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [ ]:
model2_predict_probabs

array([[0.33362615],
       [0.34413594],
       [0.52966446],
       ...,
       [0.35055757],
       [0.39623976],
       [0.4151957 ]], dtype=float32)

# Model 3 (Text Model)

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

In [ ]:
def text_preprocess(db):

  def remove_pattern(input_txt,pattern):
    import re
    r = re.findall(pattern,input_txt)
    for i in r:
      input_txt = re.sub(i,'',input_txt)
    return input_txt

  def remove_stops(row):

    my_list = row['tokenized_sents']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

  def rejoin_words(row):
    my_list = row['clean_tweet']
    joined_words = ( " ".join(my_list))
    return joined_words
    
  import string
  def remove_URL(text):
    import re
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'URL',text)
    
  def remove_HTML(text):
    import re
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
  
  def remove_not_ASCII(text):
    text = ''.join([word for word in text if word in string.printable])
    return text
  
  def word_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word
  
  def replace_abbrev(text):
    string = ""
    for word in text.split():
        string += word_abbrev(word) + " "        
    return string
  
  def remove_number(text):
    import re
    num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
    return num.sub(r'NUMBER', text)
  
  def remove_mention(text):
    import re
    at=re.compile(r'@\S+')
    return at.sub(r'USER',text)
  
  def remove_all_punct(text):
    table = str.maketrans('','',string.punctuation)
    return text.translate(table)
  
  def remove_punct(text):
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" 
    for p in punctuations:
        text = text.replace(p, f' {p} ')

    text = text.replace('...', ' ... ')
    if '...' not in text:
        text = text.replace('..', ' ... ')   
    return text
  
  def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in (stop_list)])
    return text

  def clean_tweet(text):
    text = remove_URL(text)
    text = remove_HTML(text)
    text = remove_not_ASCII(text)
    text = text.lower()
    text = replace_abbrev(text)
    text = remove_mention(text)
    text = remove_number(text)
    text = remove_punct(text)
    text = remove_stopwords(text)
    return text
  from tqdm import tqdm
  import nltk
  from nltk.corpus import stopwords
  nltk.download('stopwords', quiet=True)
  stop_list = stopwords.words('english')
  nltk.download('punkt')
  from nltk.tokenize import RegexpTokenizer
  from nltk.tokenize import word_tokenize
  import os, re, csv, math, codecs


  # For Training
  # import keras
  # from keras import optimizers
  # from keras import backend as K
  # from keras import regularizers
  # from keras.models import Sequential
  # from keras.layers import Dense, Activation, Dropout, Flatten
  # from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
  # from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
  # from keras.utils import plot_model
  # from keras.preprocessing import sequence
  # from keras.preprocessing.text import Tokenizer
  # from keras.callbacks import EarlyStopping

  # For array, dataset, and visualizing
  import numpy as np
  import re
  import pandas as pd
  import seaborn as sns
  import matplotlib.pyplot as plt
  import numpy as np
  import pandas as pd
  import seaborn as sns
  import matplotlib.pyplot as plt
  %matplotlib inline
  sns.set_style('darkgrid')

  import nltk
  from sklearn.preprocessing import LabelBinarizer
  from nltk.corpus import stopwords
  from nltk.stem.porter import PorterStemmer
  from wordcloud import STOPWORDS,WordCloud
  from nltk.stem import WordNetLemmatizer
  from nltk.tokenize import word_tokenize,sent_tokenize
  import re,string,unicodedata


  from keras.preprocessing import text,sequence
  from nltk.tokenize.toktok import ToktokTokenizer
  from string import punctuation
  from nltk import pos_tag
  from nltk.corpus import wordnet
  
  import requests, zipfile, io
  import re
  # zip_file_url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
  # r = requests.get(zip_file_url)
  # z = zipfile.ZipFile(io.BytesIO(r.content))
  # z.extractall()
  embeddings_index={}
  f= codecs.open('wiki-news-300d-1M.vec',encoding='utf-8')

  for line in tqdm(f):
    values=line.rstrip().rsplit(' ')
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeddings_index[word]=coefs
  f.close()

  print("Number of word vectors : %s"%len(embeddings_index))
  db['labels']=db['labels'].replace(['unverified'],True)
  db['labels']=db['labels'].replace(['TRUE'],True)
  db['labels']=db['labels'].replace(['FALSE'],True)
  db['labels']=db['labels'].replace(['non-rumor'],False)
  db['labels']=db['labels'].replace(['true'],True)
  db['labels']=db['labels'].replace(['false'],True)
  data=[db['text'],db['text'].apply(len),db['labels']]
  headers=["text","length","target"]
  text_df=pd.concat(data,axis=1,keys=headers)
  text_df['target']=text_df['target'].astype(int)
  targets = text_df['target'].values
  text_df['text'] = np.vectorize(remove_pattern)(text_df['text'],"#[\w]*")
  text_df['text'] = text_df['text'].str.replace("[^a-zA-Z#]", " ")
  text_df['text'] = text_df['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
  text_df['tweet'] = text_df['text'].str.lower()

  stops = set(stopwords.words('english'))

  text_df['tokenized_sents'] = text_df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
  text_df['clean_tweet'] = text_df.apply(remove_stops, axis=1)
  text_df.drop(["tweet","tokenized_sents"], axis = 1, inplace = True)
  text_df['clean_tweet'] = text_df.apply(rejoin_words, axis=1)
  abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}
  text_df['clean_tweet']=text_df['clean_tweet'].apply(clean_tweet)
  text_df["clean_tokens"] = text_df["clean_tweet"].apply(lambda x: word_tokenize(x))
  return text_df

In [ ]:
!mkdir 

In [ ]:
!cd fast_text

In [ ]:
!ls -l

total 2668569
-rw------- 1 root root  297420892 Mar 19 05:13 data.csv
drwx------ 2 root root       4096 May 21 10:37 fast_text
-rw------- 1 root root  157189018 May 10 05:20 meta_data.sav
-rw------- 1 root root       3686 May 10 04:38 meta_utils.py
drwx------ 2 root root       4096 Apr  7 17:03 __pycache__
-rw------- 1 root root   14848184 Mar 19 05:13 reactions_data.csv
-rw------- 1 root root    2532919 Mar 19 05:13 source_tweet.csv
drwx------ 2 root root       4096 May 10 07:11 tweet_user_checkpoint
-rw------- 1 root root    1504564 Mar 19 05:12 user.csv
-rw------- 1 root root      11727 Apr  8 15:06 utils.py
-rw------- 1 root root 2259088777 May 21 10:39 wiki-news-300d-1M.vec


In [ ]:
text_df=text_preprocess(tweet)

[nltk_data] Downloading package punkt to /root/nltk_data...


1104it [00:00, 11026.84it/s]

[nltk_data]   Package punkt is already up-to-date!


999995it [01:35, 10503.46it/s]


Number of word vectors : 999995


In [ ]:
text_df

,text,length,target,clean_tweet,clean_tokens
0,From jaywalking strong robbery,57,1,jaywalking strong robbery,"[jaywalking, strong, robbery]"
1,Andreas Lubitiz deemed unsuitable flight dutie...,133,1,andreas lubitiz deemed unsuitable flight dutie...,"[andreas, lubitiz, deemed, unsuitable, flight,..."
2,Classes started would there waiting eager http...,134,0,classes started would waiting eager http svaqe...,"[classes, started, would, waiting, eager, http..."
3,French says those attacked Charlie Hebdo murde...,100,0,french says attacked charlie hebdo murderers b...,"[french, says, attacked, charlie, hebdo, murde..."
4,Protest continues front police dept people uni...,136,0,protest continues front police dept people uni...,"[protest, continues, front, police, dept, peop..."
...,...,...,...,...,...
8319,photo black nurses saving life member http,79,1,photo black nurses saving life member http,"[photo, black, nurses, saving, life, member, h..."
8320,British number Andy Murray beats Australia Gro...,138,0,british number andy murray beats australia gro...,"[british, number, andy, murray, beats, austral..."
8321,minutes discussing,79,0,minutes discussing,"[minutes, discussing]"
8322,VIDEO Australian police they talking gunman ho...,118,1,video australian police talking gunman holding...,"[video, australian, police, talking, gunman, h..."


In [ ]:
X_text_train,X_text_test,y_text_train,y_text_test = train_test_split(text_df['clean_tweet'],text_df['target'],random_state=42,test_size=0.3,stratify=text_df['target'])

In [ ]:
def further_process(X_train,X_test,y_train,y_test,text_df):
  import numpy as np
  import pandas as pd
  from keras.preprocessing.text import Tokenizer
  from keras.preprocessing import sequence
  MAX_NB_WORDS=100000
  tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
  tokenizer.fit_on_texts(X_test)
  tokenizer.fit_on_texts(X_train)
  word_seq_train = tokenizer.texts_to_sequences(X_train)
  word_seq_test = tokenizer.texts_to_sequences(X_test)
  word_index = tokenizer.word_index

  max_seq_len = np.round(text_df['length'].mean()+text_df['length'].std()).astype(int)

  word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
  word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

  return word_seq_test,word_seq_train

In [ ]:
X_text_test,X_text_train = further_process(X_text_train,X_text_test,y_text_train,y_text_test,text_df)

In [ ]:
text_model = load_model('/content/drive/MyDrive/Rumor Detection/models/Text_model')

In [ ]:
y_text_test.shape[0]

2498

In [ ]:
y_text_train.shape[0]

5826

In [ ]:
result_model_3 = text_model.evaluate(X_text_test,y_text_test,verbose=2)

66/66 - 5s - loss: 0.5283 - accuracy: 0.8504
